In [ ]:
!pip install sentence-transformers gradio PyMuPDF matplotlib spacy pillow -q
!python -m spacy download en_core_web_sm -q

In [ ]:
import gradio as gr
import fitz
import numpy as np
import spacy
import matplotlib.pyplot as plt
from io import BytesIO
from PIL import Image
from sentence_transformers import SentenceTransformer, util

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")
nlp = spacy.load("en_core_web_sm")

In [ ]:

def extract_text_from_pdf(pdf_file):
    text = ""
    with fitz.open(pdf_file.name) as doc:
        for page in doc:
            text += page.get_text()
    return text.strip() if text.strip() else None

In [ ]:
def extract_skills(text):
    doc = nlp(text.lower())
    skills = set()
    for token in doc:
        if token.pos_ in ["NOUN", "PROPN"] and len(token.text) > 2:
            skills.add(token.text)
    return list(skills)


In [ ]:
def create_pie_chart(matched, missing):
    labels = ['Matched Skills', 'Missing Skills']
    sizes = [len(matched), len(missing)]
    if sum(sizes) == 0:
        sizes = [1, 1]  # prevent crash if no skills detected
    fig, ax = plt.subplots()
    ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
    ax.axis('equal')
    buffer = BytesIO()
    plt.savefig(buffer, format="png")
    plt.close(fig)
    buffer.seek(0)
    return Image.open(buffer)

In [ ]:
def analyze_resume(pdf_file, job_description):
    if pdf_file is None:
        return "No resume uploaded.", None

    resume_text = extract_text_from_pdf(pdf_file)
    if not resume_text:
        return "No text could be extracted from the resume.", None

    resume_emb = model.encode(resume_text, convert_to_tensor=True)
    job_emb = model.encode(job_description, convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(resume_emb, job_emb).item()
    similarity_percentage = round(similarity * 100, 2)

    resume_skills = extract_skills(resume_text)
    job_skills = extract_skills(job_description)
    matched_skills = [skill for skill in resume_skills if skill in job_skills]
    missing_skills = [skill for skill in job_skills if skill not in resume_skills]
    chart_img = create_pie_chart(matched_skills, missing_skills)

    result = f"Match Score: {similarity_percentage}%\nMatched Skills: {matched_skills}\nMissing Skills: {missing_skills}"
    return result, chart_img

with gr.Blocks() as demo:
    gr.Markdown("## 📌 Resume Screening System with Skills Analysis")
    with gr.Row():
        resume_input = gr.File(label="Upload Resume (.pdf)")
        job_input = gr.Textbox(label="Job Description", lines=4)
    output_text = gr.Textbox(label="Result")
    output_image = gr.Image(label="Skills Match Chart", type="pil")
    submit_btn = gr.Button("Analyze Resume")
    submit_btn.click(analyze_resume, inputs=[resume_input, job_input], outputs=[output_text, output_image])

demo.launch()
